In [6]:
import pandas as pd

#Load csv files
school_csv = "../Resources/schools_complete.csv"
student_csv = "../Resources/students_complete.csv"

#Read School and Student csv files
school_read = pd.read_csv(school_csv)
student_read = pd.read_csv(student_csv)

#Combine data files into single dataset and dataframe
school_data_df = pd.merge(student_read, school_read, how= 'left', on=['school_name', 'school_name'])
school_data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


District Summary

In [7]:
school_count = school_data_df['school_name'].nunique()
school_count


15

In [8]:
student_count = school_data_df['student_name'].value_counts()
student_total = student_count.sum()
student_total

39170

In [9]:
total_budget = school_data_df['budget'].drop_duplicates()
total = total_budget.sum()
total


24649428

In [10]:
average_math_score = school_data_df['math_score'].mean()
average_math_score

78.98537145774827

In [11]:
average_reading_score = school_data_df['reading_score'].mean()
average_reading_score

81.87784018381414

In [12]:
passing_math_count = school_data_df[(school_data_df['math_score'] >= 70)].count()['student_name']
mathPass_percent = passing_math_count / float(student_total) * 100
mathPass_percent

74.9808526933878

In [13]:
passing_reading_count = school_data_df[(school_data_df['reading_score'] >= 70)].count()['student_name']
readPass_percent = passing_reading_count / float(student_total) * 100
readPass_percent

85.80546336482001

In [14]:
overall_pass_count = school_data_df[(school_data_df['math_score'] >= 70) & 
                                    (school_data_df['reading_score']>= 70)].count()['student_name']
overall_passing = overall_pass_count / float(student_total) * 100
overall_passing

65.17232575950983

In [15]:
district_summary = [{'Total Schools': school_count, 'Total Students': student_total, 'Total Budget': total,
                     'Average Math Score': average_math_score, 'Average Reading Score': average_reading_score,
                     '% Passing Math': mathPass_percent, '% Passing Reading': readPass_percent, '% Overall Passing': overall_passing}]

district_summary = pd.DataFrame(district_summary)

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


School Summary

In [16]:
schoolType = school_data_df.set_index(['school_name'])['type']
schoolType.head()

school_name
Huang High School    District
Huang High School    District
Huang High School    District
Huang High School    District
Huang High School    District
Name: type, dtype: object

In [41]:
student_per_school = school_data_df.loc[:,['school_name']].value_counts()
student_per_school = student_per_school.rename_axis('school_name').to_frame('# of students')

student_per_school.head()


,# of students
school_name,
Bailey High School,4976
Johnson High School,4761
Hernandez High School,4635
Rodriguez High School,3999
Figueroa High School,2949


In [46]:
budget_per_school = school_data_df.groupby(['school_name']).mean()['budget']
capita_per_school = pd.merge(student_per_school, budget_per_school, on='school_name', how= 'left')
capita_per_school['per_capita_spending'] = capita_per_school['budget'] / capita_per_school['# of students']
print(capita_per_school)


                       # of students     budget  per_capita_spending
school_name                                                         
Bailey High School              4976  3124928.0                628.0
Johnson High School             4761  3094650.0                650.0
Hernandez High School           4635  3022020.0                652.0
Rodriguez High School           3999  2547363.0                637.0
Figueroa High School            2949  1884411.0                639.0
Huang High School               2917  1910635.0                655.0
Ford High School                2739  1763916.0                644.0
Wilson High School              2283  1319574.0                578.0
Cabrera High School             1858  1081356.0                582.0
Wright High School              1800  1049400.0                583.0
Shelton High School             1761  1056600.0                600.0
Thomas High School              1635  1043130.0                638.0
Griffin High School             14

In [104]:
math_per_school = school_data_df.groupby(['school_name']).mean()['math_score']
reading_per_school = school_data_df.groupby(['school_name']).mean()['reading_score']
average_tests = pd.merge(reading_per_school, math_per_school, on= 'school_name', how= 'left')
merge_average_tests = pd.merge(capita_per_school, average_tests, on= 'school_name', how= 'left')
merge_average_tests.head()


,# of students,budget,per_capita_spending,reading_score,math_score
school_name,,,,,
Bailey High School,4976,3124928.0,628.0,81.033963,77.048432
Johnson High School,4761,3094650.0,650.0,80.966394,77.072464
Hernandez High School,4635,3022020.0,652.0,80.934412,77.289752
Rodriguez High School,3999,2547363.0,637.0,80.744686,76.842711
Figueroa High School,2949,1884411.0,639.0,81.158020,76.711767


In [61]:
math_read_pass = school_data_df[(school_data_df['reading_score'] >= 70) & (school_data_df['math_score'] >= 70)]


In [109]:
pass_math_per_school = school_data_df.groupby(['school_name']).sum()['math_score']
pass_math_per_school = pass_math_per_school.rename_axis('school_name').to_frame('perc pass math')

pass_read_per_school = school_data_df.groupby(['school_name']).sum()['reading_score']
pass_read_per_school = pass_read_per_school.rename_axis('school_name').to_frame('perc pass read')
pass_mathRead_per_school = pd.merge(pass_math_per_school, pass_read_per_school, on= 'school_name', how= 'left')



pass_overall = math_read_pass.groupby(['school_name']).sum()[['reading_score', 'math_score']]
pass_overall['overall sum'] = pass_overall['reading_score'] + pass_overall['math_score']
pass_overall = pass_overall.drop(columns=['reading_score', 'math_score'])
merge_pass_overall = pd.merge(pass_mathRead_per_school, pass_overall, on= 'school_name', how= 'left')
overall_summary = pd.merge(merge_average_tests, merge_pass_overall, on= 'school_name', how= 'left')
overall_summary = overall_summary["overall sum"] / overall_summary['# of students']
overall_summary.head()

school_name
Bailey High School       92.217846
Johnson High School      90.484772
Hernandez High School    90.735059
Rodriguez High School    89.347087
Figueroa High School     89.856222
dtype: float64